In [1]:
import numpy as np
from tqdm import tqdm
import pdb 

In [2]:
#from HW02
def Readfile():
    #read binary file
    finTrainImg = open('./dataset/train-images-idx3-ubyte',mode='rb')
    finTrainLb = open('./dataset/train-labels-idx1-ubyte',mode='rb')

    #deal with training headers
    finTrainImg.seek(4,0) #first 4 bytes are the header of magic number = 0x00000803 which means the following data type is unsigned byte(meaning of 0x08) and there will be three dimensions(meaning of 0x03); to skip this header, we initially seek 4 bytes from the start of the train image file
    finTrainLb.seek(8,0) #first 4 bytes are the header of magic number. second 4 bytes are the header of first dimension, number of images, which will be defined later; to skip these headers, we initially seek 8 bytes from the start of the train lable file
    nTrainImg=int.from_bytes(finTrainImg.read(4), byteorder='big') #second 4 bytes are the header of first dimension, number of images = 0x0000ea60 = 0d60000
    #nTrainImg=100 #delete
    #finTrainImg.seek(4,1) #delete
    nRow=int.from_bytes(finTrainImg.read(4), byteorder='big') #third 4 bytes are the header of second dimension, number of rows = 0x0000001c = 0d28
    nCol=int.from_bytes(finTrainImg.read(4), byteorder='big') #forth 4 bytes are the header of third dimension, number of columns = 0x0000001c = 0d28
    
    #deal with training data and parse them
    trainImg=np.zeros((nTrainImg,nRow,nCol),dtype='uint8') #there are 60000 images; each image has 28*28 pixels; each pixel's size is 1 byte, though unit8's memory size is larger than 1 byte ... it's python's drawback
    trainLb=np.zeros(nTrainImg,dtype='uint8') #there are 60000 images, so there are also 60000 labels
    for i in tqdm(range(nTrainImg)):
        for j in range(nCol):
            for k in range(nRow):
                trainImg[i][j][k]=int.from_bytes(finTrainImg.read(1), byteorder='big') #actually, byteorder here can be either big or liitle, because there is just "1" byte    
        trainLb[i]=int.from_bytes(finTrainLb.read(1), byteorder='big')
    finTrainImg.close()
    finTrainLb.close()
    
    #binning the gray level value into two bins
    trainImg[:][trainImg[:]<=256/2]=0
    trainImg[:][trainImg[:]>256/2]=1 #don't change the bin assignment order!!! or all the pixel value will be zero
    return ((trainImg,trainLb),nTrainImg,nRow,nCol)

In [3]:
#from HW02
def Imagination(P,matchDict,modelParameter,done):
    #Imagination of number
    (fout,_,nLabel,nPixelRow,nPixelCol)=modelParameter
    
    for i in range(nLabel): #deal with each number imagination
        if done: 
            print("labeled class: ", i,file=fout)
            label=matchDict[i]
        else:
            print("unlabeled class: ", i,file=fout)
            label=i
        #mean=np.mean(P[label])
        #P[label][P[label]<mean]=0 #can't write this, or the P will be covered...QQ
        #P[label][P[label]>=mean]=1
        for x in range(nPixelRow):
            for y in range(nPixelCol):
                print((P[label][x][y]>=0.5).astype(int),end=" ",file=fout)
            print(" ",file=fout)
        print("\n",file=fout)
    

In [4]:
#from HW04 logistic regression
def ConfusionMatrix(truth,predict,number):
    matrix=np.zeros((2,2))
    matrix[0,0]=np.sum((truth==number)&(predict==number)) #TP
    matrix[0,1]=np.sum((truth==number)&(predict!=number)) #FN
    matrix[1,0]=np.sum((truth!=number)&(predict==number)) #FP
    matrix[1,1]=np.sum((truth!=number)&(predict!=number)) #TN
    return matrix

In [5]:
#from HW04 logistic regression
def ConfusionMessage(fout,truth,predict):
    for number in range(10):
        confusionMatrix=ConfusionMatrix(truth,predict,number)
        print("\n---------------------------------------------------------------\n",file=fout)
        print("Confusion Matrix {}: ".format(number),file=fout)
        print("\t\tPredict number {}\t Predict not number {}".format(number, number),file=fout)
        print("Is number {}        {:.0f}               {:.0f}       ".format(number,confusionMatrix[0,0],confusionMatrix[0,1]),file=fout)
        print("Isn't number {}     {:.0f}               {:.0f}       ".format(number,confusionMatrix[1,0],confusionMatrix[1,1]),file=fout)
        print("",end="\n",file=fout)
        sensitivity=confusionMatrix[0,0]/(confusionMatrix[0,0]+confusionMatrix[1,0]) if (confusionMatrix[0,0]+confusionMatrix[1,0])!=0 else confusionMatrix[0,0]
        specificity=confusionMatrix[0,0]/(confusionMatrix[0,0]+confusionMatrix[0,1]) if (confusionMatrix[0,0]+confusionMatrix[0,1])!=0 else confusionMatrix[0,0]
        print("Sensitivity (Successfully predict number {}): {}".format(number,sensitivity),file=fout)
        print("Specificity (Successfully predict number {}): {}".format(number,specificity),file=fout)
        print("",end="\n",file=fout)

In [6]:
def MatchTrueLabel(predictLb,trainLb,modelParameter):
    (_,_,nLabel,_,_)=modelParameter
    
    #build a table to record which fake label is more likelihood appear in true label
    table=np.zeros((nLabel,nLabel))
    for labelT in range(nLabel):
        for labelP in range(nLabel):
            table[labelT,labelP]=np.sum((trainLb==labelT)&(predictLb==labelP))
    
    #build dictionary to record true label to fake label
    matchDict = {labelT:None for labelT in range(nLabel)}
    for i in range(1, 11):
        maxNumIdx = np.unravel_index(np.argmax(table, axis=None), table.shape) #找出最大的值所在的"座標位置"，它代表著最佳match
        matchDict[maxNumIdx[0]] = maxNumIdx[1] #maxNumIdx[0] means labelT, maxNumIdx[1] means labelP

        for j in range(0,10): # record the (labelT,labelP) which we has picked
            table[maxNumIdx[0]][j] = -1 #* i
            table[j][maxNumIdx[1]] = -1 #* i
    
    #clone true label
    markPredictLb=np.zeros(nTrainImg)
    for tLabel in range(nLabel):
        markPredictLb[predictLb==matchDict[tLabel]]=tLabel
        
    return markPredictLb,matchDict

In [7]:
def EMAlogOnce(trainImg,L0,P0,modelParameter):
    (_,nTrainImg,nLabel,nPixelRow,nPixelCol)=modelParameter
    
    #----------Estep----------
    #compute the normalized responsibility based on the initial model parameter
    W=np.zeros((nLabel,len(trainImg))) #(10,60000) #version 1: no log
    logW=np.zeros((nLabel,len(trainImg))) #(10,60000) #version 2: log
    # 10*15 minutes, original version
    #for img in tqdm(range(nTrainImg)):
    #    for label in range(nLabel):
    #        for pixelRow in range(nPixelRow):
    #            for pixelCol in range(nPixelCol):
    #                logW[label][img]+=np.log(L0[label].item())+trainImg[img,pixelRow,pixelCol]*np.log(P0[label,pixelRow,pixelCol])+(1-trainImg[img,pixelRow,pixelCol])*np.log(1-P0[label,pixelRow,pixelCol]) #scalar of pixel prob
    
    
    # 30 seconds, vectorize 28*28 pixel
    #for img in tqdm(range(nTrainImg)):
    #    for label in range(nLabel):
    #        logW[label][img]=np.log(L0[label].item())+np.sum(trainImg[img,:,:]*np.log(P0[label,:,:])+(1-trainImg[img,:,:])*np.log(1-P0[label,:,:])) #scalar of image prob
    
    
    # 10 seconds, vectorize 60000 img
    minNum=1e-10 # to avoid error of log(0)
    for label in range(nLabel):
        #version 1: no log
        dataPixelProb=(trainImg[:,:,:]*P0[label,:,:]+(1-trainImg[:,:,:])*(1-P0[label,:,:])) #no log version (60000, 28, 28)
        W[label,:]=L0[label].item()*np.prod(dataPixelProb,axis=(1,2)) #no log version
        #version 2: log
        #dataPixelLogProb=trainImg[:,:,:]*np.log10(np.maximum(P0[label,:,:],minNum))+(1-trainImg[:,:,:])*np.log10(np.maximum(1-P0[label,:,:],minNum)) #log version which avoids error of log(0) version (60000, 28, 28)
        #logW[label,:]=np.log10(np.maximum(L0[label].item(),minNum))+np.sum(dataPixelLogProb,axis=(1,2)) #sum up 28*28 pixel values => (60000,)
    
    # take a look at the prediced classes
    key,value=np.unique(np.argmax(W,axis=0),return_counts=True) 
    print(dict(zip(key,value))) #np.vstack((key, value)).T
    #key,value=np.unique(np.argmax(logW,axis=0),return_counts=True) 
    #print(dict(zip(key,value))) #np.vstack((key, value)).T
    
    #normalize
    #version 1: no log
    sums=np.sum(W,axis=0) #divide by the sum of 10 labels: (10,60000) / (60000,) => (10,60000)
    sums[sums==0]=1 #to avoid divide by zero
    normW_1=W/sums #no log version
    #version 2: log
    #scaleLogW=((logW-np.min(logW)) / (np.max(logW)-np.min(logW))) * (1-minNum) + minNum #scale negative value to positive value to avoid dividing by negative value or zero ((Input - InputLow) / (InputHigh - InputLow)) * (OutputHigh - OutputLow) + OutputLow, https://stackoverflow.com/questions/51464638/what-is-the-proper-way-to-normalize-negative-values/51464692
    #normW_2=np.exp(scaleLogW)/np.sum(np.exp(scaleLogW),axis=0)
    
    #----------Mstep----------
    #update the model parameter based on the normlized responsibility
    #version 1: no log
    L1_1=(np.sum(normW_1,axis=1)/nTrainImg).reshape(-1,1) #(the sum of 60000 data / 60000).reshape(-1,1): (10,60000) => (10,1) 
    #version 2: log
    #L1_2=(np.sum(normW_2,axis=1)/nTrainImg).reshape(-1,1)
    #print(np.argsort(L1_1.reshape(-1)))
    print(L1_1.reshape(-1))
    print(np.sum(L1_1))
    
    #version 1: no log
    sums=np.sum(normW_1,axis=1) #P will sum up each probability of data, so we normalize all the data
    sums[sums==0]=1 #to avoid divide by zero
    normW_1=(normW_1.T/sums).T 
    #version 2: log
    #normW_2=(normW_2.T/np.sum(normW_2,axis=1)).T
    
    #version 1: no log
    P1_1=np.einsum('ab,bcd -> acd', normW_1, trainImg) #(10,60000) * (60000,28,28) => (10,28,28); https://stackoverflow.com/questions/58588378/how-to-matrix-multiply-a-2d-numpy-array-with-a-3d-array-to-give-a-3d-array
                                                      #another way to update P1: P1=(normLogW@trainImg.reshape(nTrainImg,nPixelRow*nPixelCol)).reshape(nLabel,nPixelRow,nPixelCol)
    #version 2: log
    #P1_2=np.einsum('ab,bcd -> acd', normW_2, trainImg)
    
    return L1_1,P1_1,W

In [9]:
if __name__=='__main__':
    #read file and deal with the bin value
    ((trainImg,trainLb),nTrainImg,nPixelRow,nPixelCol)=Readfile()
    
    #initialize model parameter
    fout=open('./dataset/out_Imagination.txt',mode='w')#sys.stdout#
    nLabel=10
    eps=10
    modelParameter=(fout,nTrainImg,nLabel,nPixelRow,nPixelCol)
    L0=np.full((nLabel,1),1/nLabel) #assume fair #np.random.rand(nLabel,1)
    P0=np.random.rand(nLabel,nPixelRow,nPixelCol) #assume fair bernoulli #np.full((nLabel,nPixelRow,nPixelCol),1/2)
    diff0,diff1=1000,1000
    done=False
    iteration=0
    while True:
        #EM algorithm
        L1,P1,logW=EMAlogOnce(trainImg,L0,P0,modelParameter)
        #print message and image
        diff0=diff1
        #version 1: no log
        diff1=np.sum(np.abs(L1-L0))+np.sum(np.abs(P1-P0))
        #version 2: log
        #diff1=np.sum(np.abs(np.log10(np.maximum(L1,1e-10))-np.log10(np.maximum(L0,1e-10))))+np.sum(np.abs(np.log10(np.maximum(P1,1e-10))-np.log10(np.maximum(P0,1e-10))))
        Imagination(P1,None,modelParameter,done)
        print("No. of Iteration: {}, Difference: {}\n".format(iteration, diff1))
        print("---------------------------------------------------------------\n")
        print("No. of Iteration: {}, Difference: {}\n".format(iteration, diff1),file=fout)
        print("---------------------------------------------------------------\n",file=fout)
        
        #determine whether to terminate
        if diff1<eps and (diff0-diff1)<eps:
            done=True
            break
        L0=L1
        P0=P1
        iteration+=1
    
    #turn unlabeled clustering into labeled clustering based on the training labels, (train label:predict label)
    predictLb=np.argmax(logW,axis=0)
    mkPredictLb,matchDict=MatchTrueLabel(predictLb,trainLb,modelParameter) #or can use another matching method : https://www.dlology.com/blog/how-to-do-unsupervised-clustering-with-keras/
    
    #print imagination
    Imagination(P1,matchDict,modelParameter,done)
    
    #print message
    ConfusionMessage(fout,trainLb,mkPredictLb)
    print("\nTotal iteration to converge: {}".format(iteration),file=fout)
    print("Total error rate: {}".format(np.sum(trainLb!=mkPredictLb)/nTrainImg),file=fout)
    
    fout.close()
        

100%|██████████| 60000/60000 [01:03<00:00, 949.14it/s] 


{0: 51841, 2: 1957, 3: 902, 4: 6, 5: 1, 6: 12, 8: 5183, 9: 98}
[1.97501863e-01 0.00000000e+00 3.25712746e-02 1.51602207e-02
 9.45736797e-05 1.66666667e-05 2.08369672e-04 0.00000000e+00
 8.67250016e-02 1.63869667e-03]
0.33391666666666664
No. of Iteration: 0, Difference: 3398.7040603857245

---------------------------------------------------------------

{0: 32090, 2: 5659, 3: 4967, 4: 93, 5: 1, 6: 253, 8: 14871, 9: 2066}
[5.33731134e-01 0.00000000e+00 9.40337335e-02 8.30249265e-02
 1.55455024e-03 1.66666667e-05 4.25502587e-03 0.00000000e+00
 2.48340642e-01 3.44766544e-02]
0.9994333333333332
No. of Iteration: 1, Difference: 219.34011375627367

---------------------------------------------------------------

{0: 29799, 2: 5199, 3: 7194, 4: 140, 5: 1, 6: 460, 8: 14574, 9: 2633}
[4.95917765e-01 0.00000000e+00 8.65827874e-02 1.19617264e-01
 3.13093087e-03 1.66666667e-05 7.68196006e-03 0.00000000e+00
 2.42556183e-01 4.39297762e-02]
0.9994333333333334
No. of Iteration: 2, Difference: 76.638435

In [ ]:
(np.sum(normW_1,axis=1)/nTrainImg)
#數量級
(Pdb)  (np.min(W),np.max(W))
(0.0, 1.0234446623722119e-290)
(Pdb)  (np.min(logW),np.max(logW))
(-387.086643795816, -289.98993563465865)
(Pdb)  (np.min(scaleLogW),np.max(scaleLogW))
(1e-10, 1.0)
#----
(Pdb)  (np.max(L1_1)-np.min(L1_1))
0.48968515325191053 -> 0.03975004604280337
(Pdb)  (np.max(L1_2)-np.min(L1_2))
0.03487774247510214 -> 0.00010147601927713856 #L1_1跟L1_2差得有點多(1e-3倍), 看來normalization with exponential並沒有讓數量級回復正常
(Pdb)  (np.max(P1_1)-np.min(P1_1))
1.0000000000000004
(Pdb)  (np.max(P1_2)-np.min(P1_2))
0.5655717808179428